In [90]:
from typing import Tuple, List

# Carregar o Léxico
Precisamos de uma lista de palavras válidas para a língua portuguesa - chamamos isso de léxico. Vamos utilizar um produzido por um time de especialistas da USP, no projeto [Unitex](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html).

Esse léxico possui mais de 75.000 palavras encontradas em textos em português. 

> ⚠ Uma análise do arquivo irá mostrar vários verbetes que não seriam válidos no term.ooo, como palavras em inglês (Windows ou wheel), pois esse léxico foi extraído de textos reais onde verbetes em outras línguas também são encontrados. Futuramente, poderíamos fazer uma redução dessa base para apenas palavras usuais.

Note que carregamos apenas palavras com 5 caracteres, uma vez que esse tamanho é fixo no `term.ooo`.

In [91]:
def load_dictionary(filename: str, word_length: int):
    """
    Loads a lexicon from a file, where the each word is the first token on a line.
    Tokens are separated by commas.
    """
    with open(filename, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.split(',')[0]
            line = line.strip()

            if len(line) == word_length:
                yield line


words = list(load_dictionary('DELAS_PB.dic', 5))

Também removemos do léxico palavras que já sabemos que não são aceitas no jogo.

In [92]:
forbidden = list(load_dictionary('forbidden_words.dic', 5))

words = list(filter(lambda word: word not in forbidden, words))

'ariel' in words


False

# Primeiro chute

Agora, vamos encontrar a palavra para ser nosso melhor primeiro "chute". Considerando uma distribuição de palavras que sigam o léxico (algo que não é, necessariamente, verdade ☹), vamos verificar quais são as letras mais frequentes. Dessa forma, vamos fazer o primeiro chute com uma palavra que contenha as letras mais frequentes na língua.

Note que as cinco letras mais frequentes podem não formar uma palavra válida. Por isto, vamos pegar a palavra do léxico que possua o maior número de letras frequentes.

Primeiro, nós contabilizamos as letras no léxico.

In [93]:
letters = {}

for word in words:
    for letter in word:
        if letter in letters:
            letters[letter] += 1
        else:
            letters[letter] = 1

# Sort by most common letter
sorted_letters = sorted(letters.items(), key=lambda x: x[1], reverse=True)
sorted_letters = list(map(lambda x: x[0], sorted_letters))


Agora, procuramos uma palavra no léxico que maximize as letras frequentes. Um ponto importante aqui é que não queremos repetições - i.e., não queremos uma palavra que tenha letras repetidas.

In [94]:
def find_word_with_letters(words: list, letters: list) -> str:
    """
    Finds the first word in the list that contains all the letters in the list.
    """
    for word in words:
        target = letters.copy()  # what letters are left to find

        for letter in word:
            if letter in target:
                target.remove(letter)

        if len(target) == len(letters) - 5:
            return word

    return None


def find_best_guess(words, ranked_letters):
    for size in range(5, len(ranked_letters)):
        letters = ranked_letters[:size]
        guess = find_word_with_letters(words, letters)

        if guess != None:
            return guess
        else:
            print("Could not find a word using the top letters {}".format(
                ''.join(letters)))


best_guess = find_best_guess(words, sorted_letters)
best_guess


Could not find a word using the top letters aorei


'arilo'

# Jogando o jogo

Com nosso primeiro chute em mãos, iniciamos o jogo. Precisamos de uma maneira para codificar a resposta que o jogo nos da para cada chute; utilizaremos uma string para isso, onde cada posição identifica o retorno do jogo, da seguinte maneira:

- Um espaço denota uma falha.
- Um caractere `i` indica que a letra está inclusa na palavra, mas não na posição correta.
- Um caracter `c` indica que a letra está inclusa na palavra e na posição correta.

Para jogar, informamos ao usuário qual deve ser o chute, e aguardamos que ele insira a resposta recebida. Aí computamos o melhor chute considerando o que já sabemos.

O melhor chute é aquele que obedece aquilo que já sabemos das palavras (por exemplo, quais letras devem estar presentes) e que prefere utilizar letras mais frequentes.

> Seguimos essa heurística por que não temos como deduzir a probabilidade de uma palavra aparecer no jogo, mas temos uma regra para determinar a probabilidade de uma palavra incluir uma letra. 

In [95]:
def validate_result(result: str) -> bool:
	if len(result) != 5:
		print('Result must be 5 letters long')
		return False
	
	if any(c not in ' ci' for c in result):
		print('Result must be composed of spaces, "i" or "c"')	
		return False

	return True

In [96]:
def describe_next_guess(included: List[str], not_included: List[str], known: List[str]) -> None:
    print('Guess must include {}'.format(', '.join(included)))
    print('Guess must not include {}'.format(', '.join(not_included)))
    print('Guess must be like {}'.format(', '.join(known)))


Para criar um novo chute, primeiro recuperamos a lista de palavras e a filtramos pelas regras.


In [97]:
def match(word: str, look_like: List[str]):
	for i in range(len(look_like)):
		if look_like[i] != ' ' and word[i] != look_like[i]:
			return False

	return True


def filter_by(words: List[str], included_letters: List[str], not_included_letters: List[str], known: List[str]):
	for word in words:
		# Check if any letter of the word is in the forbidden list
		if any(letter in not_included_letters for letter in word):
			continue
		# Check if any letter from the must-have list is missing
		if any(letter not in word for letter in included_letters):
			continue
		
		if not match(word, known):
			continue

		yield word		
	

In [98]:
def play_game(guess: str, words: List[str], ranked_letters: List[str]):
    # what letters should be in the word
    included_letters = set()

    # what letters should not be in the word
    not_included_letters = set()

    # the word that is being guessed, every letter is blank until it is known
    final_word = [letter for letter in '     ']

    while True:
        result = input(
            f'Try the guess "{guess}". What was the result? ').lower()

        if (len(result) == 0):
            print("Empty result. Stopping game.")
            return

        if not validate_result(result):
            continue

        for i, r in enumerate(result):
            letter = guess[i]

            if r == ' ':
                not_included_letters.add(letter)
            elif r == 'c':
                final_word[i] = letter
            elif r == 'i':
                included_letters.add(letter)

        describe_next_guess(included_letters, not_included_letters, final_word)

        # update list of possible words based on the current state
        words = list(filter_by(words, included_letters, not_included_letters, final_word))

        if len(words) == 0:
            print("Could not find any words that satisfy parameters. Stopping game.")

        # find the best guess
        guess = find_best_guess(words, ranked_letters)

        


play_game(best_guess, words, sorted_letters)


Result must be 5 letters long
Guess must include i
Guess must not include a, l, r
Guess must be like  ,  ,  ,  , o
Could not find a word using the top letters aorei
Could not find a word using the top letters aoreil
Could not find a word using the top letters aoreilt
Result must be 5 letters long
Guess must include i
Guess must not include l, t, r, n, a, e
Guess must be like  ,  ,  ,  , o
Could not find a word using the top letters aorei
Could not find a word using the top letters aoreil
Could not find a word using the top letters aoreilt
Could not find a word using the top letters aoreiltn
Could not find a word using the top letters aoreiltns
Could not find a word using the top letters aoreiltnsu
Could not find a word using the top letters aoreiltnsuc
Could not find a word using the top letters aoreiltnsucm
Guess must include i
Guess must not include l, d, s, t, c, r, n, a, e
Guess must be like  , i,  ,  , o
Could not find a word using the top letters aorei
Could not find a word using